In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from caafe import CAAFEClassifier  # Ensure CAAFE is correctly imported

# Load the train data
train_data = pd.read_csv('./Tasks/spaceship-titanic/train.csv')

# Handle missing values for numerical columns
imputer = SimpleImputer(strategy='median')
numerical_columns = train_data.select_dtypes(include=['float64', 'int64']).columns
train_data[numerical_columns] = imputer.fit_transform(train_data[numerical_columns])

# Convert categorical variables to numeric using OneHotEncoder
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
categorical_columns = train_data.select_dtypes(include=['object']).columns
train_encoded = encoder.fit_transform(train_data[categorical_columns])
train_encoded_df = pd.DataFrame(train_encoded, columns=encoder.get_feature_names_out())

# Combine encoded columns back to the original dataframe
train_data = train_data.drop(categorical_columns, axis=1)
train_data = pd.concat([train_data, train_encoded_df], axis=1)

# Define X and y, ensure y is not dropped from train_data prematurely
X = train_data.drop(['Transported'], axis=1)
y = train_data['Transported']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_data, y, test_size=0.2, random_state=42)

# Initialize RandomForest and CAAFE classifier
clf_no_feat_eng = RandomForestClassifier(n_estimators=100, random_state=42)
caafe_clf = CAAFEClassifier(
    base_classifier=clf_no_feat_eng,
    llm_model="gpt-4",
    iterations=2
)

# Check if the 'Transported' column is still present
print("Columns before fitting CAAFE:", X_train.columns)

# Fit the CAAFE-enhanced classifier
caafe_clf.fit_pandas(
    X_train,
    target_column_name='Transported',
    dataset_description="Predicting whether passengers are transported to another dimension based on their demographic and trip details."
)

# Predictions and evaluation
y_pred = caafe_clf.predict(X_val)
print('Validation accuracy: {:.3f}'.format(accuracy_score(y_val, y_pred)))
print('Validation classification report:')
print(classification_report(y_val, y_pred))
print('Validation confusion matrix:')
print(confusion_matrix(y_val, y_pred))

# Proceed with processing test data and predictions as previously detailed
